In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

#If cuda device exists, use that. If not, default to CPU.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [2]:
#Hyperparameters
num_epochs = 15
num_classes = 10
batch_size = 128
learning_rate = 0.001

In [3]:
#Load MNIST data set
train_dataset = torchvision.datasets.MNIST(root='./datasets/',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./datasets/',
                                          train=False,
                                          transform=transforms.ToTensor())

#Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dense1 = nn.Linear(12 * 12 * 64, 128)
        self.dense2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.dropout(x, 0.25)
        x = x.view(-1, 12 * 12 * 64)
        x = F.relu(self.dense1(x))
        x = F.dropout(x, 0.5)
        x = self.dense2(x)
        return F.log_softmax(x, dim=1)

In [5]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/15], Step [100/469], Loss: 0.3061
Epoch [1/15], Step [200/469], Loss: 0.2156
Epoch [1/15], Step [300/469], Loss: 0.1312
Epoch [1/15], Step [400/469], Loss: 0.1308
Epoch [2/15], Step [100/469], Loss: 0.0671
Epoch [2/15], Step [200/469], Loss: 0.0833
Epoch [2/15], Step [300/469], Loss: 0.0476
Epoch [2/15], Step [400/469], Loss: 0.0379
Epoch [3/15], Step [100/469], Loss: 0.0758
Epoch [3/15], Step [200/469], Loss: 0.0410
Epoch [3/15], Step [300/469], Loss: 0.1160
Epoch [3/15], Step [400/469], Loss: 0.1267
Epoch [4/15], Step [100/469], Loss: 0.1006
Epoch [4/15], Step [200/469], Loss: 0.0733
Epoch [4/15], Step [300/469], Loss: 0.0325
Epoch [4/15], Step [400/469], Loss: 0.1106
Epoch [5/15], Step [100/469], Loss: 0.1888
Epoch [5/15], Step [200/469], Loss: 0.0292
Epoch [5/15], Step [300/469], Loss: 0.0480
Epoch [5/15], Step [400/469], Loss: 0.0459
Epoch [6/15], Step [100/469], Loss: 0.0366
Epoch [6/15], Step [200/469], Loss: 0.0191
Epoch [6/15], Step [300/469], Loss: 0.0123
Epoch [6/15

In [6]:
from sklearn.metrics import roc_auc_score

preds = []
y_true = []
# Test the model
model.eval()  # Set model to evaluation mode.
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        detached_pred = predicted.detach().cpu().numpy()
        detached_label = labels.detach().cpu().numpy()
        for f in range(0, len(detached_pred)):
            preds.append(detached_pred[f])
            y_true.append(detached_label[f])

    print('Test Accuracy of the model on the 10000 test images: {:.2%}'.format(
        correct / total))

    preds = np.eye(num_classes)[preds]
    y_true = np.eye(num_classes)[y_true]
    auc = roc_auc_score(preds, y_true)
    print("AUC: {:.2%}".format(auc))
    
# Save the model checkpoint
# torch.save(model.state_dict(), 'pytorch_mnist_cnn.ckpt')

Test Accuracy of the model on the 10000 test images: 98.52%
AUC: 99.18%
